In [14]:
import re
import os
import pandas as pd
from snowballstemmer import stemmer
import arabicstopwords.arabicstopwords as ar_stp
from functools import reduce
import numpy as np
import pyterrier as pt
import torch

     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.4/7.4 MB 52.7 MB/s eta 0:00:00
     ---------------------------------------- 78.1/78.1 kB ? eta 0:00:00
     ---------------------------------------- 9.2/9.2 MB 65.9 MB/s eta 0:00:00
     --------------------------------------- 44.0/44.0 MB 54.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 93.8 MB/s eta 0:00:00
     ------------------------------------- 977.5/977.5 kB 60.5 MB/s eta 0:00:00
     ------------------------------------- 268.8/268.8 kB 16.2 MB/s eta 0:00:00
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
  Using cached packaging-23.1-py3-none-any.whl (48 kB)
     -------------------------------------- 163.8/163.8 kB 5.0 MB/s eta 0:00:00
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
     ------


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: C:\Users\gargo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for pytrec_eval, since package 'wheel' is not installed.
  Running setup.py install for pytrec_eval: started
  Running setup.py install for pytrec_eval: finished with status 'error'


  error: subprocess-exited-with-error
  
  Running setup.py install for pytrec_eval did not run successfully.
  exit code: 1
  
  [15 lines of output]
  Fetching trec_eval from https://github.com/usnistgov/trec_eval/archive/v9.0.8.tar.gz.
  C:\Users\gargo\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
    warnings.warn(
  running install
  C:\Users\gargo\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools\command\install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
    warnings.warn(
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-310
  creating build\lib.win-amd64-cpython-310\pytrec_eval
  copying py\__init__.py -> build\lib.win-amd64-cpython-310\pytrec_eval
  runn


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: C:\Users\gargo\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(0))

# Get Data

In [ ]:
data_path = "../data"
index_path = os.path.join(data_path, "QPC_Index/data.properties")

query_train_path = os.path.join(data_path, "QQA23_TaskA_train.tsv")
query_dev_path = os.path.join(data_path, "QQA23_TaskA_dev.tsv")

passage_path = os.path.join(data_path, "Thematic_QPC/QQA23_TaskA_QPC_v1.1.tsv")

qp_pair_train_path = os.path.join(data_path, "qrels\QQA23_TaskA_qrels_train.gold")
qp_pair_dev_path = os.path.join(data_path, "qrels\QQA23_TaskA_qrels_dev.gold")

## Read file

In [2]:
# read file based on its extension (tsv or xlsx)
def read_file(input_file, sep="\t", names = ""):
    if input_file.endswith(".xlsx"):
        df = pd.read_excel(input_file)
    else:
        if names != "":
            df = pd.read_csv(input_file, sep=sep, names=names,encoding="utf-8")
        else:
            df = pd.read_csv(input_file, sep=sep,encoding="utf-8")
    return df

## Read Query-Passage Pair

In [3]:
qrels_columns = ["qid", "Q0", "docid", "relevance"]

def read_qrels_file(qrels_file):
    # split_token = '\t' if format_checker.is_tab_sparated(qrels_file) else  "\s+"
    df_qrels = pd.read_csv(qrels_file, sep='\t', names=qrels_columns)
    df_qrels["qid"] = df_qrels["qid"].astype(str)
    df_qrels["docid"] = df_qrels["docid"].astype(str)
    return df_qrels

## Load the index

In [4]:
def load_index(index_path):
    if not pt.started():
        pt.init(helper_version="0.0.6")

    try:
        index = pt.IndexFactory.of(index_path)
        print("Index was loaded successfully from this path: ", index_path)
        return index
    except Exception as e:
        print('Cannot load the index, check exception details {}'.format(e))
        return []

## Cleaning
Clean text from urls, handles, special characters, tabs, line jumps, extra white space, and puntuations.

In [5]:
# Clean text from urls, handles, special characters, tabs, line jumps, and extra white space.
def clean(text):
    text = re.sub(r"http\S+", " ", text)  # remove urls
    text = re.sub(r"@[\w]*", " ", text)  # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r"\t", " ", text)  # remove tabs
    text = re.sub(r"\n", " ", text)  # remove line jump
    text = re.sub(r"\s+", " ", text)  # remove extra white space
    text = re.sub(r'[^\w\s]', '', text) # Removing punctuations in string using regex
    text = text.strip()
    return text

## Preprocessing
Preprocess the arabic input text by performing normalization, stemming, and removing stop words.

In [6]:
# arabic stemmer
ar_stemmer = stemmer("arabic")

# remove arabic stop words
def ar_remove_stop_words(sentence):
    terms=[]
    stopWords= set(ar_stp.stopwords_list())
    for term in sentence.split() : 
        if term not in stopWords :
            terms.append(term)
    return " ".join(terms)


# normalize the arabic text
def normalize_arabic(text):
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return(text)

# stem the arabic text
def ar_stem(sentence):
    return " ".join([ar_stemmer.stemWord(i) for i in sentence.split()])


# apply all preprocessing steps needed for Arabic text
def preprocess_arabic(text): 
    text = normalize_arabic(text)
    text = ar_remove_stop_words(text)
    text = ar_stem(text)
    return text

NameError: name 'stemmer' is not defined

In [7]:
def prepare_data(path, column, id_type, id_column='docno'):
        df = read_file(path, names=['docno', 'text'])

        print("Cleaning passages")
        # apply the cleaning functions on the queries/questions
        df[column] = df['text'].apply(clean)

        # apply normalization, stemming and stop word removal
        print("Preprocessing - Applying normalization, stemming and stop word removal")
        df[column] = df[column].apply(preprocess_arabic)

        df[id_type] = df[id_column].astype(str) # convert the id column to string
        df = df[[id_type, 'text', column]] # keep the columns needed for search

        print("Done with preparation!")
        return df


### Index

In [8]:
index = load_index(index_path=index_path)

print(index.getCollectionStatistics().toString())
print(index.getMetaIndex().getKeys())

# for kv in index.getLexicon():
#     print((kv.getKey())+"\t"+ kv.getValue().toString())
# index.getLexicon()["فاعل"].toString()

NameError: name 'index_path' is not defined

### Passage

In [9]:
df_passage = prepare_data(passage_path, 'passage', 'pid')
df_passage

NameError: name 'passage_path' is not defined

### Query

In [10]:
df_query_train = prepare_data(query_train_path, 'query', 'qid')
df_query_train

NameError: name 'query_train_path' is not defined

In [11]:
df_query_dev = prepare_data(query_dev_path, 'query', 'qid')
df_query_dev

NameError: name 'query_dev_path' is not defined

### Query-Passage Pair

In [12]:
df_qppair_train = read_qrels_file(qp_pair_train_path)
df_qppair_train

NameError: name 'qp_pair_train_path' is not defined

In [13]:
df_qppair_dev = read_qrels_file(qp_pair_dev_path)
df_qppair_dev

NameError: name 'qp_pair_dev_path' is not defined

# Model

## Sentence Embedding


In [17]:
# from simcse import SimSCE
from torch.utils.data import DataLoader
from torch import nn
from sentence_transformers import models, losses
from sentence_transformers import LoggingHandler, SentenceTransformer, CrossEncoder, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

import logging
from datetime import datetime
import math

In [18]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

### Retrieval

In [29]:
def save_query_passage_retrieval(result, tag, run_save=False):
    if tag == "BM25":
        result["Q0"] = ["Q0"] * len(result)
        result["tag"] = [tag] * len(result)
        result['qid'] = result["qid"]
        result['pid'] = result["docno"]
        result = result[["qid", "Q0", "pid", "rank", "score", "tag"]]

    elif tag == "SimCSE_biencoder":
        np_result = np.array(result).flatten()
        result = pd.DataFrame()

        result["qid"] = df_query_train["qid"].tolist() * top_k
        result = result.sort_values(by=['qid']).reset_index(drop=True)
        result["Q0"] = ["Q0"] * len(np_result)
        result["pid"] = [df_passage.iloc[x['corpus_id']]['pid'] for x in np_result]
        result["rank"] = list(range(1, top_k+1)) * len(df_query_train)
        result["score"] = [x['score'] for x in np_result]
        result["tag"] = [tag] * len(np_result)

    elif tag == "SimCSE_cross":
        result['tag'] = tag
        result['Q0'] = 'Q0'
        result = result[["qid", "Q0", "pid", "rank", "score", "tag"]]

    if run_save:
        run_save_path = os.path.join(data_path, f"runs/{tag}.tsv")
        result.to_csv(run_save_path, sep="\t", index=False, header=False)

    return result

###  SimCSE (Bi-Encoder)

In [20]:
#bi_model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
bi_model_name = "aubmindlab/bert-large-arabertv02" # run 0 : pAP@10 = 0.289
#bi_model_name = "wissamantoun/araelectra-base-artydiqa" # run 1 : pAP@10 = 0.437
#bi_model_name = "salti/AraElectra-base-finetuned-ARCD" # run 2 : pAP@10 = 0.397
#bi_model_name = "ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA" # run 3 : pAP@10 = 0.435
#bi_model_name = "timpal0l/mdeberta-v3-base-squad2" # run 4 : pAP@10 = 0.367
#bi_model_name = "gfdgdfgdg/arap_qa_bert" # run 5 : pAP@10 = 0.184
#bi_model_name = "gfdgdfgdg/arap_qa_bert_large_v2" # run6 : pAP@10 = 0.372
#bi_model_name = "gfdgdfgdg/arap_qa_bert_v2" # run7 : pAP@10 = 0.344
#bi_model_name = "zohaib99k/Bert_Arabic-SQuADv2-QA" # run8 : pAP@10 = 0.435
#bi_model_name = "arabi-elidrisi/ArabicDistilBERT_QA" #run 9 : pAP@10 = 0.343
#bi_model_name = "MMars/Question_Answering_AraBERT_xtreme_ar" #run 10 : pAP@10 = 0.337
# bi_model_name = "abdalrahmanshahrour/ArabicQA" # run 11 : pAP@10 = 0.304
# bi_model_name = "abdalrahmanshahrour/xtremeQA-ar" # run 12 : pAP@10 = 0.120

# model_save_path = os.path.join(data_path, f'model/training_simcse-{bi_model_name}-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

In [21]:
# Use Huggingface/transformers model (like BERT, RoBERTa, XLNet, XLM-R) for mapping tokens to embeddings
word_embedding_model = models.Transformer(bi_model_name, max_seq_length=256)
print("word_embedding_model Max Sequence Length:", word_embedding_model.max_seq_length)
print("word_embedding_model dimension", word_embedding_model.get_word_embedding_dimension())

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
print("pooling_model sentence embedding dimension", pooling_model.get_sentence_embedding_dimension())

dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.Tanh())

# bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model])
bi_encoder = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

word_embedding_model Max Sequence Length: 256
word_embedding_model dimension 1024
pooling_model sentence embedding dimension 1024
2023-08-08 18:38:11 - Use pytorch device: cuda


In [ ]:
train_batch_size = 32

In [42]:
train_samples_passage = []
#     add query train to train samples to make a more generalized embedding model
for _, row in df_passage.iterrows():
    train_samples_passage.append(InputExample(texts=[row['passage'], row['passage']]))
train_biencoder_dataloader = DataLoader(train_samples_passage, shuffle=True, batch_size=train_batch_size)

In [ ]:
train_samples_qp = []
for _, row in df_qppair_train.iterrows():
    query_id = row['qid']
    query = df_query_train[df_query_train['qid'] == query_id]['query'].tolist()[0]
    passage_id = row['docid']
    if passage_id == '-1':
        continue
    else:
        passage = df_passage[df_passage['pid'] == passage_id]['passage'].tolist()[0]
        label = row['relevance']
        #positive sample
        train_samples_qp.append(InputExample(texts=[query, passage], label=label))
        train_samples_qp.append(InputExample(texts=[passage, query], label=label))

# add query train to train samples to make a more generalized embedding model
train_qp_dataloader = DataLoader(train_samples_qp, shuffle=True, batch_size=train_batch_size)

#### Train Bi-Encoder

In [ ]:
# Configure the training
num_epochs = 10
top_k = 50

# train_biencoder_loss = losses.MultipleNegativesRankingLoss(bi_encoder)
train_biencoder_loss = losses.ContrastiveLoss(bi_encoder)
# train_biencoder_loss = losses.CosineSimilarityLoss(bi_encoder)
# train_biencoder_loss = losses.OnlineContrastiveLoss(bi_encoder)

In [43]:
warmup_steps = math.ceil(len(train_biencoder_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

# logging.info("Performance before training")
# dev_evaluator(model)

bi_encoder.fit(train_objectives=[(train_biencoder_dataloader, train_biencoder_loss)],
          # evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          # output_path=model_save_path
          )

2023-08-08 19:19:02 - Warmup-steps: 40


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

Iteration:   0%|          | 0/40 [00:00<?, ?it/s]

In [44]:
passage_embeddings = bi_encoder.encode(df_passage['passage'].tolist(), convert_to_tensor=True, show_progress_bar=True)

query_train_embeddings = bi_encoder.encode(df_query_train['query'].tolist(), convert_to_tensor=True, show_progress_bar=True)
query_dev_embeddings = bi_encoder.encode(df_query_dev['query'].tolist(), convert_to_tensor=True, show_progress_bar=True)

df_passage['embedding'] = passage_embeddings.cpu().numpy().tolist()
df_query_train['embedding'] = query_train_embeddings.cpu().numpy().tolist()
df_query_dev['embedding'] = query_dev_embeddings.cpu().numpy().tolist()

# for sentence, embedding in zip(df_passage, passage_embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [45]:
# test_samples = []
# with gzip.open(sts_dataset_path, 'rt', encoding='utf8') as fIn:
#     reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
#     for row in reader:
#         if row['split'] == 'test':
#             score = float(row['score']) / 5.0 #Normalize score to range 0 ... 1
#             test_samples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=score))
#
# model = SentenceTransformer(model_save_path)
# test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
# test_evaluator(model, output_path=model_save_path)


#### Semantic Search

In [48]:
tag = "SimCSE_biencoder"

hits = util.semantic_search(query_train_embeddings, passage_embeddings, top_k=top_k)
df_run = save_query_passage_retrieval(hits, tag, run_save=True)

print(bi_model_name, train_biencoder_loss)
! python QQA23_TaskA_eval.py \
    -r "../data/runs/SimCSE_biencoder.tsv" \
    -q "../data/qrels/QQA23_TaskA_qrels_train.gold"

df_run

aubmindlab/bert-large-arabertv02 ContrastiveLoss(
  (model): SentenceTransformer(
    (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
    (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
    (2): Dense({'in_features': 1024, 'out_features': 256, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
  )
)
Format check: Passed
    map  recip_rank
0.00847    0.021135


,qid,Q0,pid,rank,score,tag
0,101,Q0,112:1-4,1,0.205354,SimCSE_biencoder
1,101,Q0,22:19-22,2,0.194922,SimCSE_biencoder
2,101,Q0,5:116-120,3,0.193230,SimCSE_biencoder
3,101,Q0,35:19-26,4,0.189907,SimCSE_biencoder
4,101,Q0,56:27-40,5,0.185209,SimCSE_biencoder
...,...,...,...,...,...,...
8695,427,Q0,21:89-90,46,0.157940,SimCSE_biencoder
8696,427,Q0,18:78-82,47,0.157008,SimCSE_biencoder
8697,427,Q0,24:46-54,48,0.156108,SimCSE_biencoder
8698,427,Q0,5:106-108,49,0.155555,SimCSE_biencoder


### Re-ranker (Cross-Encoder)

In [49]:
# cross_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
# cross_model_name = "cross-encoder/ms-marco-TinyBERT-L-2"
cross_model_name = "distilroberta-base"

In [50]:
cross_encoder = CrossEncoder(cross_model_name)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2023-08-08 20:50:55 - Use pytorch device: cuda


#### Train Cross-Encoder

In [52]:
# Fine-tune cross-encoder on the query-passage
train_cross_loss = losses.ContrastiveLoss(cross_encoder)

torch.cuda.empty_cache()

warmup_steps = math.ceil(len(train_qp_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

cross_encoder.fit(train_qp_dataloader,
          # evaluator=dev_evaluator,
          epochs=num_epochs,
          evaluation_steps=100,
          warmup_steps=warmup_steps,
          # output_path=model_save_path
          )


2023-08-08 20:51:03 - Warmup-steps: 60


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

Iteration:   0%|          | 0/60 [00:00<?, ?it/s]

In [53]:
tag = "SimCSE_cross"

df_final = pd.DataFrame()

for qid, filter_passages in df_run.groupby('qid'):
    cross_inp = []
    q = df_query_train[df_query_train['qid'] == qid]
    for x in filter_passages.values:
        p = df_passage[df_passage['pid'] == x[2]]
        cross_inp.append([q['query'].tolist()[0], p['passage'].tolist()[0]])

    cross_scores = cross_encoder.predict(cross_inp, show_progress_bar=False)
    df_run = pd.DataFrame({'qid': qid, 'pid': filter_passages['pid'].values, 'score': cross_scores})
    df_run = df_run.sort_values(by=['score'], ascending=False)[:10]
    df_run['rank'] = range(1, len(df_run) + 1)

    df_final = pd.concat([df_final, df_run], ignore_index=True)

df_final = save_query_passage_retrieval(df_final, tag, run_save=True)

print("cross-encoder model name", cross_model_name)
! python QQA23_TaskA_eval.py \
    -r "../data/runs/SimCSE_cross.tsv" \
    -q "../data/qrels/QQA23_TaskA_qrels_train.gold"
df_final

cross-encoder model name distilroberta-base
Format check: Passed
     map  recip_rank
0.005904    0.030252


,qid,Q0,pid,rank,score,tag
0,101,Q0,42:20-22,1,0.999807,SimCSE_cross
1,101,Q0,7:54-56,2,0.999807,SimCSE_cross
2,101,Q0,4:163-166,3,0.999807,SimCSE_cross
3,101,Q0,42:27-31,4,0.999807,SimCSE_cross
4,101,Q0,42:1-6,5,0.999807,SimCSE_cross
...,...,...,...,...,...,...
1735,427,Q0,49:1-5,6,0.999806,SimCSE_cross
1736,427,Q0,66:10-12,7,0.999806,SimCSE_cross
1737,427,Q0,43:74-80,8,0.999806,SimCSE_cross
1738,427,Q0,7:44-45,9,0.999806,SimCSE_cross


In [40]:
! python QQA23_TaskA_eval.py \
    -r "../data/runs/SimCSE_cross.tsv" \
    -q "../data/qrels/QQA23_TaskA_qrels_train.gold"

Format check: Passed
    map  recip_rank
0.01003    0.045361


## BM25 - Search
Search in the index and find the relevant passages.

In [39]:
# initialize the BM25 retrieval model
BM25_model = pt.BatchRetrieve(index, controls = {"wmodel": "BM25"}, num_results=top_k)
# wmodel="TF_IDF"

In [40]:
tag = "BM25"

# search using BM25 model
df_run = BM25_model.transform(df_query_train)

# save the run in trec format to a file
df_run = save_query_passage_retrieval(df_run, tag, run_save=True)

df_run

,question-id,Q0,passage-id,rank,score,tag
0,101,Q0,7:85-93,0,14.236550,BM25
1,101,Q0,29:36-37,1,13.002848,BM25
2,101,Q0,11:89-95,2,12.651979,BM25
3,101,Q0,11:84-88,3,11.658893,BM25
4,101,Q0,26:176-191,4,6.080667,BM25
...,...,...,...,...,...,...
6503,427,Q0,3:199-200,45,6.774431,BM25
6504,427,Q0,6:19-21,46,6.773260,BM25
6505,427,Q0,5:15-17,47,6.742466,BM25
6506,427,Q0,15:1-9,48,6.715582,BM25


In [41]:
sample = "عمران"
BM25_model.search(sample)

,qid,docid,docno,rank,score,query
0,1,1115,66:10-12,0,8.576149,عمران
1,1,116,3:33-41,1,8.129249,عمران


In [ ]:
def batch_emb(l1, l2):
    l2 = sentence_embedding.encode(l2)
    l1.extend(l2)
    return l1

def get_embedding(text, n=50):
    batched_num = [text[i:i + n] for i in range(0, len(text), n)]
    texts_embed = reduce(batch_emb, batched_num, [])
    print(len(texts_embed))
    return texts_embed



## RocketQA

# Evaluation

In [42]:
! python QQA23_TaskA_eval.py \
    -r "../data/runs/bigIR_BM25.tsv" \
    -q "../data/qrels/QQA23_TaskA_qrels_dev.gold"

Format check: Passed
     map  recip_rank
0.170291    0.313333


Here, we are just evaluating the perfect run for the dev set

In [ ]:
! python QQA23_TaskA_eval.py \
    -r "../data/runs/dev_perfect.tsv" \
    -q "../data/qrels/QQA23_TaskA_qrels_dev.gold"